# 6

### 1번의 문제점 해결을 위해 BPE를 활용 방법 사용

### "직거래"라는 단어가 사용된 경우만 사용하여 장소 리스트 생성

In [37]:
import pandas as pd
from kiwipiepy import Kiwi
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
import matplotlib.pyplot as plt
import numpy as np
import os
import datetime
from statsmodels.formula.api import ols
from matplotlib import rc

In [38]:
pd.options.display.max_colwidth = 1000

In [40]:
df = pd.read_excel('./df_token_2.xlsx')

In [57]:
df = df.dropna(axis=0)

In [66]:
df_직거래 = df[df.detail.str.contains('직거래',na=False)]

In [67]:
df_직거래.to_csv('./직거래.csv',encoding='utf-8-sig')

In [68]:
df = pd.read_csv("직거래.csv")
df = df.drop(['Unnamed: 0'],axis=1)
df.head(2)

,num,region,title,category,time,price,detail,interest,chat,view,upload_datetime,upload_date,upload_ym,reup,region_gu,token,token_sum,cate_temp,cate_final
0,207637769,강남구 논현동,(가격인하)내열유리 커피잔 4인조세트,생활/가공식품,7달 전,12000,"선물받고 한번도 사용하지 않은 제품입니다. 6인조인데 2개는 사용해서 4인조 세트 판매합니다.\nmade in czech 체코생산제품으로 \n정말 가볍습니다.\n최종인하가격입니다.\n학동역, 논현초 직거래.",9,0,179,2021-02-28 23:59:59,2021-02-28 00:00:00,2021-02,0,강남구,['커피잔'],커피잔,['dishes'],dishes
1,191491540,강남구 압구정동,<미개봉> 버섯선물세트,생활/가공식품,8달 전,20000,"이번추석에 선물받은 버섯선물세트입니다. 표고,영지,상황,차가버섯 입니다. 버섯을 잘못먹어서 저렴하게 내놓습니다~!\n압구정고등학교 근처 직거래 원합니다.",2,2,116,2021-01-30 23:59:59,2021-01-30 00:00:00,2021-01,0,강남구,['버섯'],버섯,['fruits'],fruits


In [70]:
class Seperate:
    def __init__(self, df):
        self.df = df
     
    def this_month(self):
        return self.df[self.df.time.str.contains("초|분|시간|일")].reset_index(drop=True)

In [71]:
a = Seperate(df)
this_month = a.this_month()
this_month.shape

(3292, 19)

In [72]:
# 한달 내 자료로 detail 단어 학습 

with open ('sample.txt', 'w', encoding='utf8') as f:
    for row in this_month.itertuples():
        try:
            f.write(row.detail)
            f.write('\n')
        except:
            pass
        
from tokenizers import CharBPETokenizer
bpe = CharBPETokenizer(lowercase=True)
bpe.train(files='sample.txt', min_frequency=1, vocab_size=30000) # article 수가 21037개 이므로 넉넉하게 주었다. 

In [73]:
# BPE를 활용한 토크나이저 함수 
# 길이가 1 이상인 것만 추출 
def tokenizer(text):
    enc = bpe.encode(text)
    for token in enc.tokens:
        token = token.replace('</w>', '')
        if len(token) > 1:
            yield token

# 키위 토크나이저 
kiwi = Kiwi()
def extract_noun(text):
    result = kiwi.analyze(text)
    for lemma, pos, _, _ in result[0][0]:
        if pos.startswith('N'):
            yield lemma
            
def compare(num):
    text = this_month.detail[num]
    print('article detail:' , text)
    print('-'*60)
    print('bpe tokenizer:', list(tokenizer(text)))
    print('kiwi tokenizer:', list(extract_noun(text))) 

In [74]:
compare(803)

article detail: CJ 특별한선택 H-3호 스팸 선물세트
난곡사거리/신대방역 직거래
------------------------------------------------------------
bpe tokenizer: ['cj', '특별한선택', '3호', '스팸', '선물세트', '난곡사거리', '신대방역', '직거래']
kiwi tokenizer: ['특별', '선택', '호', '스팸', '선물', '세트', '난곡사거리/신대', '방역', '직거래']


In [75]:
compare(718)

article detail: 보라매역 직거래 가능
1세트에 1만원
------------------------------------------------------------
bpe tokenizer: ['보라매역', '직거래', '가능', '1세트에', '1만원']
kiwi tokenizer: ['보라매역', '직거래', '가능', '세트', '만', '원']


In [76]:
cv = CountVectorizer(max_features=10000, tokenizer=tokenizer)
dtm = cv.fit_transform(this_month.detail)

wc = pd.DataFrame({'단어': cv.get_feature_names(), '빈도':dtm.sum(axis=0).flat})
wc.sort_values('빈도', ascending=True).head(20)

,단어,빈도
9999,🥕서울대입구역,1
7689,카드지갑과,1
7690,카드지갑에는,1
4515,영등포로,1
4514,영등포구청역쪽에서,1
7691,카드지갑은,1
7692,카드할인가로,1
7693,카디건,1
4509,영도교회,1
4508,영남중학교,1


In [82]:
# cv 많이쓰인 단어 10000개
b = []
for i in cv.get_feature_names():
    b.append(i)
print(b)

['00', '000', '000대입니다', '000에', '000원', '000원에', '000원이고', '000원이네요', '000원이에요', '000원인데', '000원입니다', '000원짜리', '000으로', '01', '01까지', '01로', '01일까지', '02', '03', '04', '04월', '05', '06', '06월', '07', '08', '10', '100', '10000원', '10000원에', '1000원', '100g', '100ml', '100mm', '100퍼센트', '100호', '101동', '106', '108호', '109', '10kg', '10ml', '10p', '10개', '10개까지', '10개씩', '10만원', '10번', '10번출구', '10번출구에서', '10분거리', '10시', '10월', '10호', '11', '110', '115g', '117', '11단지', '11만원에', '11번', '11번출구', '11월', '11호', '12', '120', '12000원', '12000원에', '120g', '120g×2개', '125ml', '128', '12ml', '12m이지만', '12개', '12개가', '12개씩', '12개월', '12개입', '12길13', '12만원', '12시', '12월', '12입니다', '12캔', '12호', '13', '13000', '13000원', '13000원에', '135g', '14', '14000원에', '14개', '14번', '14일', '14일까지입니다', '15', '1500', '15000', '15000원', '15000원에', '15000원이고', '150g', '150ml', '15ml', '15개', '15만원', '15일', '15일까지', '16', '1600', '16000원에', '1600원', '16cm', '16개', '16일', '16일까지입니다', '16편수', '16호', '17', '17000원', '17

### 손으로 매칭

In [109]:
location_dict = {}
location_dict['station'] = ['가디역','가락시장역','가산디지털단지역','가산디지털역','가양역','강남구청역','강남역','강동구청역','강동역','강변역','개롱역','개봉역','개봉역','개화산역','거여역','건대역','건대입구역','경복궁역','경전철북한산보국문역','경찰병원역','고려대역','고속터미널역','공덕역','공릉역','공항시장역','광운대역','광화문역','광흥창역','교대역','구디역','구로디지털단지역','구로역','구반포역','구산역','구역삼','구의역','구일역','구파발역','국회의사당역','군자역','굽은다리역','금천구청역','금호역','길동역','길음역','김포공항역','까치산역','낙성대역','남구로역','남부터미널역','남성역','남영역','내방역','노들역','노량진역','노원역','녹번역','녹천역','논현역','논현역','답십리역','당산역','당산역','대림역','대모산입구역','대방역','대청역','대치역','대흥역','도곡역','도봉산역','도봉역','독립문역','독산역','돌곶이역','동대문역','동묘앞역','등촌역','디엠씨역','뚝섬역','뚝섬유원지역','마곡나루역','마곡역','마들역','마장역','마포구청역','마포대흥역','마포역','망우역','망원역','매봉역','먹골역','면목역','명일역','목동역','몽촌토성역','무악재역','문래역','문정역','미사역','미아사거리역','미아역',
 '반포역','발산역','방배역','방이역','방학역','방화역','보라매역','보문역','봉은사역','봉천역','북한산보국문역','불광역','사당역','사평역','삼각지역','삼성역','삼성중앙역','삼양역','삼전역','상계역','상도역','상봉역','상봉역','상왕십리역','상일동역','새내역','새절역','샛강역','서대문역','서울대입구역','서울숲역','서울역','서초방배역','서초역','석계역','석촌고분역','석촌역','선릉역','선유도역','선정릉역','성수역','성신여대역','솔샘역','송정역','송파나루역','송파역','수락산역','수서역','수유역','숭실대역','숭실대입구역','시장역','신길역','신논현역','신대방삼거리역','신대방역','신도림역','신림역','신목동역','신반포역','신방화역','신사역','신설동역','신용산역','신정네거리역','신정역','신촌기차역','신촌역','신풍역','쌍문역','아차산역','아현역','안국역','암사역','압구정로데오역','압구정역','애오개역','양재시민의숲역','양재역','양천구청역','양천향교역','양평역','어린이대공원역','언주역','여의나루역','여의도역','역곡역','역삼역','역촌역','연신내역','염창역','영등포구청역','영등포시장역','영등포역','오금역','오류동역','오류역','오목교역','옥수역','온수역','온수역점','올림픽공원역','왕십리역','외대앞역','외지역','용답역','용두역','용산역''우장산역','월계역','월곡역','위역','응암역','이대역','이수역','이촌역','이태원역','일원역',
 '잠실나루역','잠실새내역','잠실역','잠원역','장승배기역','장지역','장한평역','제기동역','종각역','종합운동장역','중계역','중랑역','중화역','증미역','증산역','창동역','창신역','천왕역','천호역','철산역','청담역','청량리역','총신대입구역','충무로역','충정로역','태릉입구역','하계역','학동역','한성대입구역','한성백제역','한티역','합정역','행당역','혜화역','홍대입구역','홍입역','홍제역','화계역','화곡역','화랑대역','효창공원앞역','효창공원역','효창운동장역','흑석역']
location_dict['entrance'] = ['경전철','개찰구','개찰구에서','버스정거장','버스정류장','버스정류장에서','신월7동강월초입구사거리버스정거','신월7동경창빌딩버스정거장','신정네거리지하철3번출구앞','고속버스터미널','고속터미널','정류장','지하철','지하철로','지하철만','지하철요금','지하철이용']
location_dict['home'] = ['저희집앞에서','저희집에','저희집으로','저희집은','저희집쪽에서만','집근처','집근처나','집근처에서','집까지','집니다','집도','집만','집안','집압','집압직거래','집앞','집앞까지','집앞비대면','집앞에서','집앞으로','집앞으로오','집앞직거래','집앞직거래가능해요','집앞직거래또는','집앞직거래만가능','집앞직거래만합니다','집에','집에도','집에만','집에서','집에서는','집에서도','집으로','집은','집이','집이라','가락쌍용','가락프라자아파트앞','개봉푸르지오','공덕4차','관악드림타운','길음뉴타운','더현대','도봉한신아파트직거래가능합니다','독산동롯데캐슬1차','독산동롯데캐슬1차직거래가능합니다',
 '롯데캐슬','래미안','동작상떼빌근처','백련산힐스테이트','문정레미안','삼성래미안','벽산아파트에서','삼풍아파트에서','상계주공','상계주공2단지','아파트','아파트103동','아파트근처가','아파트근처입니다','아파트나','아파트단지에서','아파트로','아파트상가','아파트앞','아파트에서','아파트입니다','아파트점','아파트정문에서','아파트쪽에서','서래마을','송파위례','신대방현대아파트입니다','신안약수아파트101동','신정이펜하우스','아이파크','압구정한양현대','연희파크푸르지오','용산e편한세상','용산e편한세상에서','용산파크자이',
 '우성3차','우성4차','우성5차','용산더프라임','월곡동아','위례17단지나','위례송파푸르지오','위례신도시','은평뉴타운','정릉풍림아이원에서','청계산','텐즈힐2단지','포레나','포레스트','하계동삼익선경아파트직거래','하계동삼익선아파트직거래','헬리오시티','헬리오시티서','헬리오시티에서','헬리오직거래','현대아파트에서','현대불광','청계벽산','효창파크푸르지오',
 '휴먼시아','마곡엠밸리','관악푸르지오','가락프라자아파트','대림벽산아파트','대림아파트','도봉한신아파트','두산아파트','두산위브아파트','목동1단지아파트','벽산아파트','삼익아파트','삼풍아파트','신대방현대아파트','신안약수아파트','양재우성아파트','엘스아파트','오금삼성아파트','오아파트','우성아파트','우아파트','은마아파트','잠실엘스아파트','주공아파트','진도아파트','촌아파트','풍전아파트','하계동삼익선경아파트','하계동삼익선아파트','한양수자인아파트','한진아파트','행당대림아파트','현대아파트','홍은벽산아파트','화곡대림아파트']
location_dict['park'] = [ '공원','공원에서','올림픽공원쪽이나','거리공원','고척근린공원','교체어린이공원','근린공원','보라매공원','신트리공원앞','안말공원근처','양재근린공원','어린이공원','어린이대공원','올림픽공원','정훈공원','지공원','효심어린이공원','효창공원']
location_dict['gucenter'] = ['구로구청앞','동사무소','묵동한국전력공사','성동구청에서','성동우체국','성북세무소나','오류동우체국','염창동사무소','도서관에서','고척도서관에서','용답우체국앞','용산우체국','우체국','우체국앞이구요','우체국에','응암1동사무소','직거래ㅡ강동구청','직거래ㅡ방학2동주민센터','개봉3동자치센타','아트센타','주민센터근처','주민센터앞','주민센터앞이나','주민센터에서','후암동사무소앞에서','주민센타','독산4동주민센터','동주민센터','보라매동주민센터','삼각산동주민센터','센터피스','송파산모건강증진센터','아리랑시네센터','안전센터','주민센터','중계1동주민센터','방학2동주민센터','천호2동주민센터','체육센터','치안센터','휘경1동주민센터''주민센타','독산4동주민센터','동주민센터','보라매동주민센터','삼각산동주민센터','센터피스','송파산모건강증진센터','아리랑시네센터','안전센터','주민센터','중계1동주민센터','방학2동주민센터','천호2동주민센터','체육센터','치안센터','휘경1동주민센터','강남구청','강동구청','강서구청','구로구청','구청','금천구청','성동구청','양천구청','영등포구청','은평구청','강동구청']
location_dict['medical'] = [ '강남성심병원근처에서','구로보건소','문정약국에서','강남성심병원','강동성심병원','병원','보라매병원','새움병원','서부병원','아산병원','양지병원','여성병원','차병원','메디칼약국', '문정약국', '약국', '진약국앞']
location_dict['hotel'] = ['공군호텔','나이아가라호텔','더데이호텔','신라호텔','안호텔','해군호텔','호텔','힐탑호텔','더데이호텔앞','호텔에서','호텔쪽입니다']
location_dict['church'] = [ '강남교회','교회','교회앞','산정현교회','성당','암사동성당','암사성당','양광교회','연신교회','영도교회','쑥고개성당','왕십리교회','왕십리교회앞에서','잠원성당','화곡본동성당서','흰돌교회앞']
location_dict['school'] = [ '낙성대','대학교','덕성여대','명지대','명지전문대','상문고','서경대근처','서울대','서울대입구','숙대입구','세종대','성신여대','숙대','어린이집','어린이집근처에서','연신초교앞','어린이집이나','연대앞','양천고에서','오금중학교정문','외대근처에서','외대근처에요','외대앞','외대정문직거래입니다','용산고','월정초등학교근처','월정초등학교나','월정초에서','정신여고','초교쪽에서','초근처','초등','초등학','초등학교','초등학교3ㅡ4','초등학교나','초등학교앞에','초등학교에서','초등학교정문앞에서','초앞','초에','홍대','경문고','고등학교','삼각산고''상문고','서울고','압구정고등학교','양천고','진비고','참송고','현대고등학교','휘문고','흑화고','문성유치원', '유치원', '화계유치원','남서울중학교','삼각산중','삼성중앙역','송파중학교','신길중','양서중','여중','역삼중','영남중학교','오금중학교','오류중','오류중학교','원묵중','월곡중학교','중학교','강동초','강서초','길원초','논현초','대조초등학교','서초역','석촌초','숭례초','양동초등학교','어성초','역삼초','역촌초','연신초교','염경초','염리초','염창초','영문초','영화초등학교','오봉초','온곡초','우장산초','우장초등학교','월정초','잠전초등학교','천동초','청원초','초당초등학교','태릉초','화양초등학교','휘경초등학교']
location_dict['store'] = [ '뚜레쥬르','롯데리아','롯데마트','롯데마트에서','롯데백화점','롯데백화점에','롯데백화점에서','롯데캐슬상가로','롯백','마트','마트나','마트서','마트에서','망원시장','동부시장','더바디샵','다이소','다이소앞','다이소앞에서','다이소에서','백화점','백화점서','백화점에서','방이시장','봉일시장','삼성디지털프라자','상봉이마트','상가','상가앞에서','성대시장','세븐일레븐','수유시장','신세계','신세계백화점','신성시장','신원시장','아모레','아모레퍼시픽','아모페퍼시','아울렛에서','아름할인마트','엔씨백화점','엔씨백화점입니다','올림픽훼밀리','왕십리텐즈힐몰이나','우성상가','우림시장','월계이마트','웰빙마트','은평이마트','이마트','이마트나','이마트에서','장미제일시장근처','텐즈힐몰','하남돼지집','풍천장어','현대아울렛에서','현대백화점','현대백화점에서','현대시장','현대3차상가','현대3차상가내','홈플','홈플러스','홈플러스에서','현백','후암시장근처나','후암시장에서']
location_dict['etc'] = [ '개봉사거리','경희궁','구의사거리','기업은행','길동사거리','까치산지구대','남부터미널','난곡사거리','다목적광장에서','농협','답십리사거리','당곡사거리','대림삼거리','동대문구','동방사우나','미아사거리','북가좌사거리','로데오거리','목동사거리','삼각지','삼거리','산맥빌딩','석촌호수','송리단길','신대방삼거리','시조사삼거리','신림로','수유대로','신정네거리','아차산사거리입니다','원효로','오거리','오금동사거리','올림픽대교','우리랜드','우리은행','월정로','위레광장','은행사거리에서','은행','전농동사거리','지하주차장','청담사거리','학동사거리','청림농원','장위동고개','화양사거리나','화양삼거리','사거리','한티','강서경찰서', '경찰서', '광진경찰서', '서부경찰서', '용산경찰서', '종암경찰서','고척도서관','공군회관','도서관','등빛도서관','마포중앙도서관','송파여성문화회관','양천문화회관','여성회관','천호도서관','해군회관','해맞이도서관','회관']
location_dict['gu'] = ['종로구','중구','용산구','성동구','광진구','동대문구','중랑구','성북구','강북구','도봉구','노원구','은평구','서대문구','마포구','양천구','강서구','구로구','금천구','영등포구','동작구','관악구','서초구','강남구','송파구','강동구']
location_dict['dong'] = ['봉천동', '신원동', '서원동', '신림동', '은천동', '신사동', '난곡동', '청룡동', '행운동','남현동', '인헌동', '청림동', '대학동', '중앙동', '보라매동', '성현동', '낙성대동', '조원동','미성동', '난향동', '서림동', '중화동', '면목본동', '묵동', '상봉동', '중화제1동', '상봉제2동','신내2동', '망우동', '면목제2동', '면목동', '신내동', '상봉제1동', '신내1동', '면목제7동','묵제1동', '망우제3동', '망우본동', '묵제2동', '중화제2동', '면목제3.8동', '연희동', '창천동','북아현동', '현저동', '홍제제3동', '홍은동', '북가좌동', '충정로3가', '옥천동', '홍은제1동','홍제제1동', '홍제동', '대현동', '북가좌제2동', '북가좌제1동', '신촌동', '홍은제2동', '충현동','남가좌동', '충정로2가', '천연동', '남가좌제1동', '대신동', '남가좌제2동', '방화제1동','화곡제1동', '화곡동', '마곡동', '염창동', '내발산동', '등촌동', '등촌제3동', '방화동','화곡본동', '화곡제3동', '방화제2동', '발산제1동', '가양동', '화곡제6동', '가양제3동','화곡제4동', '등촌제1동', '등촌제2동', '우장산동', '화곡제2동', '공항동', '석촌동', '잠실7동','문정2동', '거여2동', '잠실2동', '위례동', '잠실3동', '가락동', '문정동', '방이2동', '삼전동','잠실본동', '가락2동', '송파동', '가락1동', '오금동', '송파2동', '문정1동', '풍납2동','장지동', '거여동', '방이동', '잠실동', '잠실6동', '풍납동', '잠실4동', '거여1동', '송파1동','신천동', '구로동', '개봉제3동', '오류동', '온수동', '개봉제1동', '구로제3동', '고척동','구로제2동', '구로제5동', '수궁동', '오류제1동', '신도림동', '항동', '천왕동', '개봉동','오류제2동', '개봉제2동', '구로제1동', '고척제2동', '궁동', '고척제1동', '구로제4동', '자양동','구의제1동', '구의제3동', '화양동', '구의동', '중곡제2동', '능동', '군자동', '중곡제4동','구의제2동', '자양제2동', '자양제3동', '자양제1동', '중곡동', '광장동', '중곡제3동', '중곡제1동','자양제4동', '서초2동', '서초동', '반포3동', '잠원동', '방배본동', '내곡동', '서초4동','방배동', '우면동', '반포4동', '양재동', '방배3동', '반포2동', '반포동', '방배4동', '서초1동','반포본동', '서초3동', '방배2동', '반포1동', '방배1동', '양재1동', '양재2동', '창동','도봉동', '도봉제1동', '쌍문제4동', '방학제1동', '창제5동', '도봉제2동', '쌍문동', '창제1동','방학동', '창제2동', '방학제3동', '쌍문제2동', '방학제2동', '창제4동', '창제3동', '쌍문제1동','마장동', '상왕십리동', '금호동1가', '성수1가제2동', '금호1가동', '성수동2가', '금호동3가','도선동', '응봉동', '성수동1가', '행당동', '금호동2가', '하왕십리동', '성수2가제3동','왕십리제2동', '금호4가동', '성수1가제1동', '옥수동', '성수2가제1동', '행당제2동', '용답동','행당제1동', '금호2.3가동', '금호동4가', '왕십리도선동', '사근동', '논현동', '논현2동','삼성1동', '수서동', '개포4동', '역삼동', '대치동', '도곡1동', '역삼1동', '대치1동','일원본동', '개포동', '대치4동', '청담동', '논현1동', '삼성동', '도곡동', '도곡2동', '자곡동','압구정동', '일원동', '일원2동', '세곡동', '삼성2동', '역삼2동', '대치2동', '응암동','신사제1동', '불광제1동', '역촌동', '진관동', '구산동', '대조동', '녹번동', '불광동','응암제2동', '갈현제2동', '응암제1동', '수색동', '갈현제1동', '불광제2동', '증산동', '갈현동','응암제3동', '신사제2동', '부사동', '태평2동', '산성동', '문화동', '용두동', '문화1동','태평1동', '안영동', '태평동', '대흥동', '은행선화동', '선화동', '문화2동', '석교동', '문창동','목동', '유천동', '대사동', '사정동', '중촌동', '유천1동', '유천2동', '은행동', '중계2.3동','상계8동', '하계2동', '공릉2동', '중계동', '공릉동', '상계2동', '상계동', '월계동', '상계9동','월계3동', '하계동', '중계4동', '월계2동', '중계본동', '공릉1동', '상계1동', '상계6.7동','상계3.4동', '중계1동', '상계5동', '하계1동', '월계1동', '상계10동', '고덕제1동', '천호동','길동', '명일제2동', '명일동', '천호제1동', '고덕제2동', '강일동', '상일동', '암사제2동','둔촌제2동', '암사제1동', '성내제3동', '성내동', '암사제3동', '둔촌동', '명일제1동', '성내제1동','암사동', '천호제2동', '성내제2동', '고덕동', '문래동3가', '영등포동7가', '신길동', '당산동4가','양평동2가', '신길제6동', '대림제2동', '여의도동', '문래동2가', '대림제1동', '영등포동4가','당산동1가', '영등포동5가', '영등포동8가', '영등포본동', '신길제1동', '대림동', '양평제1동','문래동6가', '여의동', '양평동1가', '도림동', '영등포동3가', '문래동', '양평동4가', '당산동6가','양평제2동', '당산동5가', '신길제7동', '당산동2가', '당산동3가', '양평동3가', '당산동','신길제3동', '대림제3동', '당산제2동', '영등포동1가', '양평동5가', '가산동', '시흥동','독산제3동', '독산동', '시흥제4동', '시흥제3동', '시흥제1동', '시흥제2동', '독산제1동','독산제2동', '독산제4동', '신월2동', '신월7동', '목5동', '신정7동', '신월동', '신월6동','목4동', '신정1동', '신정4동', '목2동', '신정2동', '신정6동', '신정동', '신정3동','신월5동', '신월1동', '목3동', '목1동', '신월3동', '신월4동', '상도동', '대방동', '노량진동','노량진제1동', '사당동', '사당제5동', '사당제1동', '사당제2동', '상도제3동', '본동','노량진제2동', '동작동', '사당제3동', '신대방동', '흑석동', '상도제1동', '신대방제2동','상도제2동', '상도1동', '사당제4동', '상도제4동', '한강로2가', '동자동', '한강로1가', '한남동','이촌동', '도원동', '이태원동', '용산동2가', '이촌제1동', '동빙고동', '후암동', '원효로1가','원효로제1동', '산천동', '한강로3가', '청파동', '용문동', '용산동3가', '이태원제2동', '한강로동','원효로제2동', '용산동6가', '청파동1가', '서계동', '보광동', '이태원제1동', '효창동', '원효로3가','갈월동', '청파동3가', '서빙고동', '용산동1가', '용산동5가', '신계동', '남영동', '망원동','아현동', '공덕동', '합정동', '상수동', '신수동', '망원제1동', '연남동', '창전동', '상암동','성산제1동', '성산동', '서교동', '서강동', '신공덕동', '성산제2동', '동교동', '토정동', '염리동','도화동', '노고산동', '용강동', '중동', '혜화동', '삼청동', '원서동', '숭인동', '관수동','창신동', '홍파동', '명륜2가', '동숭동', '평창동', '교남동', '창신제2동', '익선동', '창신제3동','신문로1가', '무악동', '연지동', '평동', '계동', '옥인동', '세종로', '교북동', '이화동','통인동', '공평동', '청운효자동', '내수동', '명륜4가', '종로1.2.3.4가동', '사직동','숭인제2동', '명륜1가', '행촌동', '수송동', '당주동', '종로1가', '연건동', '청진동', '가회동','신문로2가', '중학동', '송현동', '신영동', '구기동', '적선동', '인의동', '종로2가', '효자동','서린동', '청량리동', '이문동', '장안제2동', '전농제1동', '답십리동', '장안동', '전농동','제기동', '용신동', '장안제1동', '답십리제1동', '회기동', '휘경동', '이문제2동', '휘경제2동',
       '전농제2동', '답십리제2동', '신설동', '휘경제1동', '번동', '수유동', '미아동', '번1동','우이동', '수유1동', '수유2동', '삼각산동', '송중동', '송천동', '수유3동', '인수동', '번2동','번3동', '삼양동', '월곡제2동', '종암동', '동소문동6가', '길음동', '하월곡동', '장위동','보문동6가', '안암동', '돈암동', '성북동', '정릉제4동', '정릉제3동', '장위제3동', '상월곡동','석관동', '정릉동', '정릉제2동', '삼선동2가', '동선동', '삼선동3가', '돈암제1동', '길음제2동','삼선동', '보문동4가', '정릉제1동', '장위제1동', '동선동2가', '안암동1가', '동소문동4가','성북동1가', '돈암제2동', '보문동2가', '길음제1동', '삼선동1가', '망원제2동', '마포동','홍제제2동', '신길제5동', '영등포동', '신길제4동', '원효로2가', '문배동', '원효로4가', '개포2동','일원1동', '풍납1동', '오륜동', '가락본동', '마천1동', '방이1동', '마천동', '동선동4가','문래동4가', '송정동', '방화제3동', '가양제1동', '시흥제5동', '신대방제1동', '가양제2동','현석동', '보문동7가', '안암동5가', '양평동6가', '면목제4동', '가리봉동', '송월동', '누상동','홍지동', '삼선동5가', '보문동', '월곡제1동', '필운동', '종로5.6가동', '신교동', '명륜3가','통의동', '재동', '효제동', '부암동', '관훈동', '종로6가', '안국동', '종로3가', '홍익동','옥계동', '영등포동2가', '당산제1동', '면목제5동', '쌍문제3동', '신창동', '동선동3가','천호제3동', '마천2동', '영천동', '미근동', '화곡제8동', '율현동', '영등포동6가']
location_dict['door'] = [ '정문','정문거래','정문앞','정문앞에서','정문에서','정문입니다','출구','출구가능합니다','출구로','출구에서','출구와','출구이고','출구이시고','출구입니다','출구직거래요','후문','후문앞','후문에서','후문이나','후문쪽']
# 직접 = ['가락2동', '가락쌍용', '가락프라자아파트앞','가산동', '가산디지털단지', '가양', '가재울', '갈산', '갈현동', '강남', '강남교회', '강남구', '강남성심병원근처에서', '강동', '강동구', '강동리버', '강변', '강북', '강서구','강일', '개롱역부근', '개롱역에서','개봉', '개봉3동', '개봉3동자치센타',  '개봉동', '개봉사거리', '개봉역에서','개봉푸르지오','개찰구', '개찰구에서', '개포동', '개포동인근', '개화산역1번출구','건대입구','경전철', '경희궁','고속버스터미널','고속터미널', '고속터미널역앞','고양','고척도서관에서', '고척동','골목', '공덕', '공덕4차', '공덕역이나', '공릉동','공영', '공원', '공원에서', '공진', '공항동','관악', '관악구', '관악드림타운', '관악점', '관앞에서', '광명', '광운대역에서', '광장동', '광주요', '광진구', '광화문','교대역에서','교회', '교회앞','구로', '구로구', '구로구청앞', '구로보건소', '구로역에서', '구룡', '구리','구반포','구산동', '구성','구의', '구의동인근직거래희망입니다', '구의사거리', '구파발', '구파발역에서','군자', '군자동','굽은다리역에서','금호동입니다💛','기업은행', '기에', '길동', '길동사거리','길음뉴타운', '길음동', '길음역에서','까치산', '까치산지구대','낙성대', '낙성대역에서', '난곡', '난곡사거리', '남가좌', '남구로', '남문', '남부법원쪽', '남부터미널','노량진','노원', '노원구', '노티', '녹번역근처','논현', '논현동', '논현역에서','농협','다목적광장에서','다이소', '다이소앞', '다이소앞에서', '다이소에서','답십리', '답십리사거리','답십리역점', '당곡사거리','당산', '당산역에서','대도식당에서','대림동', '대림삼거리', '대모산입구역에서', '대방동', '대방역or마포역', '대방역에서','대천', '대청역에서','대치동', '대치동에서', '대학교', '대학동', '더데이호텔앞', '더바디샵','더현대', '덕성여대','도곡역에서','도림동', '도봉1동', '도봉산역이나', '도봉한신아파트직거래가능합니다', '도서관에서', '도화동', '도화동에서', '독립문', '독막로20나길','독산', '독산1동', '독산3동', '독산고개', '독산동', '독산동롯데캐슬1차', '독산동롯데캐슬1차직거래가능합니다', '독산동지점', '독산본동','돈암', '돈암동', '돌곶이역에서','동대문구','동방사우나','동부시장', '동부이촌동', '동사무소','동작구', '동작상떼빌근처','두산위브앞에서','둔촌2동', '둔촌동','등촌', '등촌역에서','뚜레쥬르','뚝섬역2번출구', '뚝섬유원지역입니다','래미안','로데오거리','롯데리아', '롯데마트', '롯데마트에서', '롯데백화점', '롯데백화점에', '롯데백화점에서', '롯데캐슬', '롯데캐슬상가로', '롯백', '마곡', '마곡나루', '마곡역직거래원합니다','마들역부근입니다','마장','마트', '마트나', '마트서', '마트에서', '마티', '마포', '마포구', '마포역4출', '마포점','망원', '망원동', '망원시장', '명일동','명지대', '명지전문대','목2동', '목4동','목동', '목동사거리','몽촌토성','무악재역에서','묵동', '묵동한국전력공사','문래', '문래동', '문래동점', '문배동','문정', '문정동', '문정동에서', '문정레미안', '문정약국에서','미아', '미아사거리', '미아사거리역2번출구입니다','발산', '발산역에서','방배3동', '방배동직거래합니다', '방배본동', '방배카페골목','방이동', '방이시장','방학', '방학역3번출구나', '방화동에서', '백련산힐스테이트','백화점', '백화점서', '백화점에서','버스정거장', '버스정류장', '버스정류장에서','번2동이나', '번3동이요','벽산아파트에서','봉일시장', '봉제산쪽입니다', '봉천동', '봉천역에서', '부근', '부근에서', '부근에서만', '부근이나', '부근이에요', '부근입니다','북가좌사거리', '북서울꿈의숲','사근동','사당', '사당방향', '사당역에서', '사당역직거래', '사당역직거래합니다','사봉','사평','산맥빌딩','산정현교회','삼각산고', '삼각지', '삼거리', '삼부', '삼선sk', '삼성', '삼성동', '삼성디지털프라자', '삼성래미안', '삼성역에서', '삼육', '삼전동','삼풍아파트에서','상가', '상가앞에서', '상계dt점', '상계동', '상계역1', '상계역에서', '상계주공', '상계주공2단지','상도동', '상문고', '상봉', '상봉동', '상봉역에서', '상봉역이나', '상봉이마트','상암동', '상왕십리', '상월곡','상일동', '상일동역에서','서경대근처', '서교동', '서대문', '서래마을','서문','서울', '서울대', '서울대입구', '서울직거래원해요', '서원', '서일', '서초구', '서초동', '석관동', '석관동에서','석촌', '석촌동', '석촌호수', '선릉','성남', '성내2동', '성내3동', '성내동', '성당', '성대시장', '성동구', '성동구청에서', '성동우체국', '성북레포츠', '성북세무소나','성산동', '성산동에서', '성산점', '성수', '성수2가', '성수동', '성신여대', '성신여대입구역', '성신여대입구역에서', '성원', '성인', '성축', '세계', '세곡', '세네번안에', '세무서','세븐일레븐','세종대','송리단길', '송이', '송정', '송천염', '송파', '송파2동', '송파구', '송파위례', '송파하비오','수유', '수유3동', '수유대로', '수유시장','숙대', '숙대입구', '숭인동','시조사삼거리','시흥1동', '시흥4동', '시흥동','신금호', '신길6동', '신길7동', '신길동', '신길역이나', '신내1동', '신내동', '신논현', '신논현역에서', '신대방삼거리', '신대방삼거리역에서', '신대방삼거리역입니다', '신대방역에서', '신대방현대아파트입니다', '신도림', '신림', '신림로', '신림역에서', '신림역이나보라매공원근처', '신림직거래', '신반포', '신방화', '신사', '신사동', '신사양', '신성시장', '신세계', '신세계백화점', '신안약수아파트101동', '신앙촌', '신오브제주', '신용산', '신원시장', '신월', '신월2동', '신월6동', '신월6동강신중하차', '신월6동쌍용자동차대리점앞', '신월7동', '신월7동강월초입구사거리버스정거', '신월7동경창빌딩버스정거장', '신월동', '신정3동', '신정네거리', '신정네거리지하철3번출구앞', '신정동', '신정이펜하우스','신진', '신창', '신촌', '신파크','쌍문', '쌍문동에서', '쌍문삼익세라믹', '쌍용예가', '쌍용플래티넘','쑥고개', '쑥고개성당', '쑥고개입구','아름제일', '아름할인마트', '아리랑고개', '아리스톤', '아리차이', '아마', '아머', '아모레', '아모레퍼시픽', '아모페퍼시','아울렛에서','아이파크','아차산사거리입니다', '아차산역근처', '아차산역근처에서', '아차산역에서', '아차산역이나', '아트리치', '아트몰링', '아트센타', '아티', '아파트', '아파트103동', '아파트근처가', '아파트근처입니다', '아파트나', '아파트단지에서', '아파트로', '아파트상가', '아파트앞', '아파트에서', '아파트입니다', '아파트점', '아파트정문에서', '아파트쪽에서','아현', '아현동','안동회사에서', '안방학동','암사', '암사3동입니다', '암사동', '암사동성당', '암사성당', '암사역근처나', '암사역에서', '암사역이나', '암압', '암웨이', '압구정', '압구정동', '압구정로데오역에서', '압구정점서', '압구정한양현대','애오개', '애오개역1번출구에서', '애오개역에서', '애오개역이나', '애오개역쪽에서', '양광교회','양재', '양재2동입니다', '양재동', '양재시민의', '양재우성점', '양재천','양천고에서', '양천구', '양천벤처', '양천향교','양화','어린이대공원역직거래', '어린이집', '어린이집근처에서', '어린이집이나','엔씨백화점', '엔씨백화점입니다', '엔씨직거래나','엘스동문','여의나루역입니다', '여의도', '역2번출구', '역근처', '역근처에서', '역근처입니다', '역도', '역말사거리', '역삼', '역삼2동', '역삼4호', '역삼남지점', '역삼동', '역삼럭키', '역삼역근처', '역삼역에서', '역삼역이','역에서', '역에서만', '역이고', '역이나', '역인근에서', '역입니다', '역직거래', '역직거래합니다', '역쪽에서', '역촌동', '역촌사거리', '역촌사거리부근', '역촌역이나', '연남동', '연남점','연대앞','연습장','연신교회', '연신내', '연신내역에서', '연신내역이나', '연신초교앞','연희동', '연희파크푸르지오','염창', '염창동', '염창동사무소', '염창동입니다', '염창역이나','영도교회','영등포', '영등포구', '영등포구청역쪽에서', '영등포로', '영등포역은', '영등포역입니다','영암', '영천','영화사','옆집', '오거리', '오금동', '오금동사거리', '오금역7번출구', '오금역이나', '오금중학교정문','오류동', '오류동에서', '오류동역에서', '오류동우체국', '오목교', '오목교역에서', '오목교역이나', '온수골', '온수골사거리', '온수동', '온수동직거래','올림픽공원쪽이나', '올림픽대교','올림픽훼밀리','완도산','왕십리교회', '왕십리교회앞에서', '왕십리나', '왕십리본점', '왕십리역에서', '왕십리역입니다', '왕십리텐즈힐몰이나','외대근처에서', '외대근처에요', '외대앞', '외대정문직거래입니다', '외발산동','용답동', '용답동까진', '용답역에서', '용답우체국앞','용두동', '용두점에서','용문', '용산', '용산e편한세상', '용산e편한세상에서', '용산고', '용산구', '용산구내', '용산더프라임', '용산에서', '용산역', '용산역아니면', '용산우체국', '용산파크자이','우리랜드','우리은행','우림시장', '우면동', '우방', '우성3차', '우성4차', '우성5차', '우성상가','우장산', '우장산역', '우장산역에서', '우장산역이나', '우장상','우체국', '우체국앞이구요', '우체국에','원효', '원효로', '원흥', '월계', '월계2동', '월계동', '월계미성', '월계이마트', '월곡동아', '월곡역이구요', '월곡역직거래', '월드', '월배', '월세트입니다', '월요일', '월이', '월이상', '월정로', '월정초등학교근처', '월정초등학교나', '월정초에서', '월촌','웰빙마트','위레광장', '위례', '위례17단지나', '위례송파', '위례송파푸르지오', '위례신도시', '위례직거래','은천', '은평구', '은평뉴타운', '은평이마트', '은행', '은행사거리에서','응봉', '응암', '응암1동사무소', '응암동','이마트', '이마트나', '이마트에서','이수', '이수역직거래','이촌', '이촌동','이태원', '이태원565', '자양동','잠실', '잠실나루역근처에서', '잠실동', '잠실새내', '잠실에서','잠원', '잠원동', '잠원성당','장미제일시장근처','장안2동', '장안동', '장위1동', '장위동', '장위동고개','장지', '장한평역에서','저희집앞에서', '저희집에', '저희집으로', '저희집은', '저희집쪽에서만','전농1동직거래또는택배가능', '전농동', '전농동사거리','정류장', '정릉풍림아이원에서','정문', '정문거래', '정문앞', '정문앞에서', '정문에서', '정문입니다','정신여고','종암동','주민센터근처', '주민센터앞', '주민센터앞이나', '주민센터에서','중계', '중계동', '중계본동', '중계역에서','중곡', '중곡2동', '중곡3동', '중곡4동', '중곡동', '중국', '중랑구', '중랑역1번출구', '중랑역에서', '중화동','증산역이나','지하주차장', '지하철', '지하철로', '지하철만', '지하철역', '지하철역까지는', '지하철역에서', '지하철요금', '지하철이용','직거래ㅡ강동구청', '직거래ㅡ방학2동주민센터', '직거래ㅡ왕십리역', '직거래ㅡ화곡분소','직거래ㆍ답십리역','직거래개롱역이에요','직거래남성역','직거래불광역', '집근처', '집근처나', '집근처에서', '집까지', '집니다', '집도','집만','집안', '집압', '집압직거래', '집앞', '집앞까지', '집앞비대면', '집앞에서', '집앞으로', '집앞으로오', '집앞직거래', '집앞직거래가능해요', '집앞직거래또는', '집앞직거래만가능', '집앞직거래만합니다', '집에', '집에도', '집에만', '집에서', '집에서는', '집에서도', '집으로', '집은', '집이', '집이라','창동', '창동역에서','천연동','천왕', '천왕역근처', '천왕역이에요', '천왕역입니다', '천원', '천원대구요', '천원씩', '천원이', '천원정도', '천으로', '천일염', '천지랜드', '천지양홍삼진액', '천천히', '천호', '천호2동', '천호동', '천호동고분', '천호역가능해요', '천호역에서', '천호역입니다','청계벽산', '청계산','청년회의소', '청담', '청담cg', '청담동', '청담사거리', '청담역에서', '청량리', '청량리나', '청량리역2번출구입니다', '청량리역에서', '청림농원', '청림동','초교쪽에서', '초근처', '초등', '초등학', '초등학교', '초등학교3ㅡ4', '초등학교나', '초등학교앞에', '초등학교에서', '초등학교정문앞에서', '초록마을','초앞', '초에','출구', '출구가능합니다', '출구로', '출구에서', '출구와', '출구이고', '출구이시고', '출구입니다', '출구직거래요','텐즈힐2단지', '텐즈힐몰','토토부동산앞에서','포레나', '포레스트','풍천장어','하계동', '하계동삼익선경아파트직거래', '하계동삼익선아파트직거래', '하계동입니다', '하계세이브존앞에서요', '하계역2번출구', '하계역에서', '하계쪽에서','하고선정릉','하남돼지집','학동사거리', '학여울','헬리오시티', '헬리오시티서', '헬리오시티에서', '헬리오직거래','현대', '현대3차상가', '현대3차상가내', '현대능동', '현대백화점', '현대백화점에서', '현대벤쳐', '현대불광', '현대시장', '현대아울렛에서', '현대아파트에서', '현대약품', '현대월드', '현대하이','현백','호텔에서', '호텔쪽입니다','홈플', '홈플러스', '홈플러스에서','홍대', '홍대입구', '홍대입구역에서','홍은동', '홍제동입니다', '홍제역2번출구', '홍제천','화곡', '화곡1동', '화곡1동에서', '화곡2', '화곡2동', '화곡4동', '화곡6동', '화곡8동', '화곡동', '화곡동에서', '화곡본동', '화곡본동성당서', '화곡부기어린이', '화곡역3번출구', '화곡역근처', '화곡역까지', '화곡역에서', '화곡역직거래', '화랑대역이나','화양동', '화양리', '화양사거리나', '화양삼거리','효자동', '효자동입니다', '효창동', '효창파크푸르지오', '후문', '후문앞', '후문에서', '후문이나', '후문쪽','후암동', '후암동사무소앞에서', '후암동에서', '후암동종점', '후암시장근처나', '후암시장에서','휘경2동', '휘경동', '휘경브라운스톤','휴먼', '휴먼시아','흰돌교회앞','패션문화', '의거리','️합정','🌿수서','👉약수역','👉직거래ᆢ화곡역3번출구','📍석촌역', '📍어린이대공원역','🔥대치역', '🥕서울대입구역']


all_list = []
for i in list(location_dict.values()):
    for j in i:
        all_list.append(j)

len(all_list)

1497

In [104]:
zone = [a for a in b if "경찰" in a] 
zone

['강서경찰서', '경찰병원역', '경찰서', '용산경찰서', '종암경찰서']

In [105]:
# 딕셔너리 활용 
def tokenizer_in_dict(text):
    enc = bpe.encode(text)
    for token in enc.tokens:
        token = token.replace('</w>', '')
        if token in all_list:
#             if len(token) > 1:
            yield token
                
def compare_bpe(num):
    text = this_month.detail[num]
    print('article detail:' , text)
    print('-'*60)
    print('bpe tokenizer:', list(tokenizer(text)))
    print('bpe+dict tokenizer:', list(tokenizer_in_dict(text)))

In [106]:
compare_bpe(666)

article detail: 🔸직거래 우장초등학교 정문 or 문성유치원
------------------------------------------------------------
bpe tokenizer: ['직거래', '우장초등학교', '정문', 'or', '문성유치원']
bpe+dict tokenizer: ['우장초등학교', '정문', '문성유치원']


In [107]:
compare_bpe(408)

article detail: 직거래 희망위치는 미아사거리역, 삼각산중, 삼각산고 입니다.
------------------------------------------------------------
bpe tokenizer: ['직거래', '희망위치는', '미아사거리역', '삼각산중', '삼각산고', '입니다']
bpe+dict tokenizer: ['미아사거리역', '삼각산중']


In [108]:
len(this_month)

3292

### all_list 안에 있는 단어가 문장 안에 있으면 그때마다 리스트에 넣는 함수

In [90]:
good = []
for i in range(3292): #this_month 개수 = 15264
    text = this_month.detail[i]
    for j in tokenizer(text):
        if j in tokenizer_in_dict(text):
            good.append(j)
print(good)

['개포동', '양재근린공원', '현대백화점', '정문입니다', '강남구청역', '선정릉역', '백화점', '청담사거리', '강남역', '선릉역', '매봉역', '저희집은', '역삼역', '선정릉역', '압구정역', '강남역', '강남구청역', '한티역', '학동역', '현대백화점에서', '현대백화점', '도곡역', '풍납동', '강남구청역', '신사역', '논현역', '봉은사역', '역삼역', '한티역', '역삼역', '출구에서', '청담동', '개포동', '롯백', '역삼역', '일원1동', '주민센터에서', '강남구청역', '선릉역', '대치동', '집에서', '집에서', '일원역', '논현동', '집에서', '자곡동', '사거리', '대청역', '강남구', '논현동', '저희집쪽에서만', '역삼역', '매봉역', '도곡역', '선릉역', '수서역', '저희집은', '대치동', '강남구청역', '학동사거리', '봉은사역', '신세계백화점', '강남구', '압구정역', '신사역', '강남구청', '매봉역', '학동역', '논현초', '집에', '강남구청역', '일원1동', '주민센터에서', '신라호텔', '삼풍아파트', '신촌동', '강남구청역', '보라매동주민센터', '당곡사거리', '신림역', '대청역', '강남구청역', '압구정로데오역', '삼성역', '대치4동', '선릉역', '집에', '서울숲역', '압구정역', '한티역', '강남역', '역삼역', '선릉역', '삼성디지털프라자', '양재근린공원', '역삼동', '개포동', '언주역', '역삼초', '신사동', '삼성동', '언주역', '학동역', '강남구', '논현동', '저희집쪽에서만', '강남구청역', '학동사거리', '집에', '역삼2동', '압구정역', '삼성중앙역', '집에만', '압구정한양현대', '현백', '집에서', '낙성대역', '강남구청역', '매봉역', '도곡동', '매봉역', '집에', '신논현역', '논현초', '학동역', '개포동', '청담동', '학동사거리', '롯데백